In [30]:
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

In [31]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test.csv')

In [32]:
class_encoding = {'CAG': 0, 'NAG': 1, 'OAG': 2}
train_df['label'] = train_df['class_label'].map(class_encoding)
test_df['label'] = test_df['class_label'].map(class_encoding)

In [33]:
train_df = train_df[["comment", "label"]]
test_df = test_df[["comment", "label"]]

In [34]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [35]:
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'label'],
        num_rows: 75
    })
    test: Dataset({
        features: ['comment', 'label'],
        num_rows: 90
    })
})

In [37]:
model_ckpt = 'ai4bharat/indic-bert'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, keep_accents=True)

In [38]:
def tokenize(batch):
    return tokenizer(batch['comment'], padding='max_length', truncation=True, max_length=512)

In [39]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

### Our classifier

In [40]:
import torch 
from transformers import AutoModel
model_ckpt = 'ai4bharat/indic-bert'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [41]:
print(model.config)

AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "vocab_size": 200000
}



In [42]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return{'hidden_state':last_hidden_state[:,0].cpu().numpy()}
    
dataset_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
dataset_hidden_states = dataset_encoded.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [43]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [44]:
import torch 
from transformers import AutoModel
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
# device = torch.device('cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [45]:
dataset_hidden_states

DatasetDict({
    train: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 75
    })
    test: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 90
    })
})

In [46]:
import numpy as np
# Where hidden states will be the inputs and labels are the targets
X_train = np.array(dataset_hidden_states['train']['hidden_state'])
X_valid = np.array(dataset_hidden_states['test']['hidden_state'])
y_train = np.array(dataset_hidden_states['train']['label'])
y_valid = np.array(dataset_hidden_states['test']['label'])
X_train.shape, X_valid.shape

((75, 768), (90, 768))

In [47]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train,y_train)
round(lr_clf.score(X_valid,y_valid),3)

0.4

In [48]:
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train)

svm_score = svm_clf.score(X_valid, y_valid)
print(f"SVM accuracy score: {round(svm_score, 3)}")

SVM accuracy score: 0.389


In [49]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

rf_score = rf_clf.score(X_valid, y_valid)
print(f"Random Forest accuracy score: {round(rf_score, 3)}")

Random Forest accuracy score: 0.556


In [50]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

In [51]:
from sklearn.metrics import accuracy_score, f1_score
def perf_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1 score': f1}

In [52]:
from transformers import Trainer, TrainingArguments
batch_size = 5 
logging_steps = len(dataset_encoded['train']) // batch_size
model_name = 'Custom Text Classifier'
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=10, 
                                  learning_rate=3e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy='epoch',
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level='error')

In [53]:
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=perf_metrics,
                  train_dataset=dataset_encoded['train'],
                  eval_dataset=dataset_encoded['test'],
                  tokenizer=tokenizer)
trainer.train()

/Users/vishals/miniconda3/envs/Tensorflow_Env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 1.0987, 'grad_norm': 0.9182733297348022, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.0}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.098726749420166, 'eval_accuracy': 0.3111111111111111, 'eval_f1 score': 0.1778105539167486, 'eval_runtime': 8.8786, 'eval_samples_per_second': 10.137, 'eval_steps_per_second': 2.027, 'epoch': 1.0}
{'loss': 1.0989, 'grad_norm': 1.2672005891799927, 'learning_rate': 2.4e-05, 'epoch': 2.0}


...

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.0885225534439087, 'eval_accuracy': 0.37777777777777777, 'eval_f1 score': 0.32484472778932294, 'eval_runtime': 12.5602, 'eval_samples_per_second': 7.166, 'eval_steps_per_second': 1.433, 'epoch': 9.0}
{'loss': 1.0555, 'grad_norm': 2.099292039871216, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.0860443115234375, 'eval_accuracy': 0.37777777777777777, 'eval_f1 score': 0.3138916042141849, 'eval_runtime': 11.1875, 'eval_samples_per_second': 8.045, 'eval_steps_per_second': 1.609, 'epoch': 10.0}
{'train_runtime': 357.048, 'train_samples_per_second': 2.101, 'train_steps_per_second': 0.42, 'train_loss': 1.085343100229899, 'epoch': 10.0}


TrainOutput(global_step=150, training_loss=1.085343100229899, metrics={'train_runtime': 357.048, 'train_samples_per_second': 2.101, 'train_steps_per_second': 0.42, 'train_loss': 1.085343100229899, 'epoch': 10.0})

In [54]:
preds_output = trainer.predict(dataset_encoded['test'])
preds_output.metrics

  0%|          | 0/18 [00:00<?, ?it/s]

{'test_loss': 1.0860443115234375,
 'test_accuracy': 0.37777777777777777,
 'test_f1 score': 0.3138916042141849,
 'test_runtime': 11.0024,
 'test_samples_per_second': 8.18,
 'test_steps_per_second': 1.636}